In [1]:
import os
import sys

# Only the TensorFlow backend supports string inputs.
os.environ["KERAS_BACKEND"] = "tensorflow"

import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from keras import layers

sys.path.append(os.path.abspath("../../src/"))
import mapping_fits_objects as mfo
from data_acquisition_understanding.dnn_datasets_preparation import read_and_concat_catalogs, clean_and_split_catalog

2025-03-26 15:37:38.195837: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-26 15:37:38.255654: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loading and preparing the datasets

Reading and merging catalog and mapping files

In [2]:
columns = ["OBJECT_ID", "FITS_ID", "CCD_ID", "ISO0", "BACKGROUND", "ELLIPTICITY", "ELONGATION", "CLASS_STAR", "FLAGS", "EXPTIME"]
data_path = "../../data/"
proc_path = os.path.join(data_path, "processed")
fm_path = os.path.join(data_path, "for_modeling")


In [3]:
catalog_paths = [
        os.path.join(fm_path, "objects_catalog_cadc2.parquet.gz"),
        os.path.join(fm_path, "objects_catalog_ngc0869.parquet.gz"),
        os.path.join(fm_path, "objects_catalog_ngc0896.parquet.gz"),
        os.path.join(fm_path, "objects_catalog_ngc7000.parquet.gz")
    ]

catalog = read_and_concat_catalogs(catalog_paths) 

In [4]:
catalog_cleaned = catalog.dropna(subset=["gt_label1"])
catalog_cleaned = catalog_cleaned.drop(columns=["gt_label2", "OBJECT_ID"])

In [6]:
train_df, val_df, test_df, class_weights = clean_and_split_catalog(
    catalog,
    "gt_label1",
    ["gt_label2", "OBJECT_ID"],
    0.6,
    0.5,
    42
)

Class weights:
{'GOOD': 0.5751560667765035, 'BT': 0.206570084977698, 'RBT': 0.1958864105401491, 'B_SEEING': 0.01747255955803839, 'BGP': 0.004914878147610943}
-----------------


/home/mike/shared-drives/C:/Users/FournigaultM/git/computational_astro/astro_iqa/src/data_acquisition_understanding/dnn_datasets_preparation.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[col] = train_df[col].astype(str)
/home/mike/shared-drives/C:/Users/FournigaultM/git/computational_astro/astro_iqa/src/data_acquisition_understanding/dnn_datasets_preparation.py:74: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_df[col] = val_df[col].astype(str)
/home/mike/shared-drives/C:/Users/Fournigaul

In [ ]:
# verifying all the classes are present in the datasets
class_weights_train = train_df["gt_label1"].value_counts(normalize=True)
class_weights_train = class_weights_train.to_dict()
print("Class weights train:")
print(class_weights_train)
print("-----------------")
class_weights_val = val_df["gt_label1"].value_counts(normalize=True)
class_weights_val = class_weights_val.to_dict()
print("Class weights val:")
print(class_weights_val)
print("-----------------")
class_weights_test = test_df["gt_label1"].value_counts(normalize=True)
class_weights_test = class_weights_test.to_dict()
print("Class weights test:")
print(class_weights_test)
print("-----------------")